In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import pickle
import time
import dill
#http://www.newyorksocialdiary.com/party-pictures?page=1
#url = "http://www.newyorksocialdiary.com/party-pictures"
#response = requests.get(url, params={"limit": 1000, "offset": 0})
#response = requests.get(url,params={'page':3})
#print response.url
#print response.text[:200] + "..."

In [2]:

def scrap1(soup):
    parent_div = soup.select('div[class=view-content]')[0] #Find (at most) *one*
    a=parent_div.select('span[class=field-content]')
    #print a
    N=len(a)
    event=[a[i].text for i in range(N) if (i % 2)==0]
    date=[a[i].text for i in range(N) if (i % 2)==1]
    link=[a[i].select('a')[0]['href'] for i in range (N) if (i%2)==0]
    df=pd.DataFrame({'event':event,'date':date,'link':link})
    return (df)


df=pd.DataFrame(columns=['event','date','link'])
for page in range (27):
    response = requests.get("http://www.newyorksocialdiary.com/party-pictures",params={'page':page})
    soup = BeautifulSoup(response.text,"lxml")
    ans=scrap1(soup)
    df=pd.concat([df,ans])

#Updating index

df.date=pd.DatetimeIndex(df.date)
df.index=df.date

before=datetime.strptime("30/11/14", "%d/%m/%y")
df=df.ix[before:]
df

    


,date,event,link
date,,,
2014-11-24,2014-11-24,Gala Guests,/party-pictures/2014/gala-guests
2014-11-20,2014-11-20,Equal Justice,/party-pictures/2014/equal-justice
2014-11-18,2014-11-18,Celebrating the Treasures,/party-pictures/2014/celebrating-the-treasures
2014-11-17,2014-11-17,Associates and Friends,/party-pictures/2014/associates-and-friends
2014-11-13,2014-11-13,Michael's 25th,/party-pictures/2014/michaels-25th
2014-11-12,2014-11-12,New York Lifelines,/party-pictures/2014/new-york-lifelines
2014-11-10,2014-11-10,Legends and Leaders,/party-pictures/2014/legends-and-leaders
2014-11-06,2014-11-06,Fall Fun,/party-pictures/2014/fall-fun
2014-11-05,2014-11-05,Park Avenue Armory's 2014 Gala Masquerade,/party-pictures/2014/park-avenue-armorys-2014-...


In [4]:
links=["http://www.newyorksocialdiary.com" + df.link[i] for i in range(len(df))]

In [6]:
def caption_list (url): #Retrieves url, returns a list of photo captions
    x=0
    while x<10: # Trying 10 max for each URL before exiting
        response = requests.get(url)
        if response.status_code==200:
            break
        else:
            x+=1
    if x==10:
        return []
    
    soup = BeautifulSoup(response.text)
    captions_list=[]
    
    captions = soup.select('div[class=photocaption]')  #First caption type
    if len(captions)>0:
        captions_list=[captions[i].text for i in range (len(captions))]
    else:
        captions = soup.findAll("font", {"face": "Verdana, Arial, Helvetica, sans-serif"}) #Second caption type
        if len(captions)>0:
            captions_list=[captions[i].text for i in range (len(captions))]
        else:
            captions = soup.findAll("td", {"class": "photocaption","scope":"row"}) #Third caption type
            if len(captions)>0:
                captions_list=[captions[i].text for i in range (len(captions))]
            else: 
                captions = soup.select('td[class=photocaption]') #Fourth caption type
                if len(captions)>0:
                    captions_list=[captions[i].text for i in range (len(captions))]
    
    if len(captions_list)>0:
        return captions_list
    else:
        print url
        return []
    

d=[] #the caption containers
counter=0
for url in links:
    time.sleep(0.1)
    if len(caption_list(url))>0:
        counter+=1
        d.extend(caption_list(url))
    
    if counter%10==0: #Printing and saving files
        print(counter)
        outfile=open( str(counter+1), "wb" )
        #dill.dump(d, outfile)
        outfile.close()
    #a=all_captions + caption_list(url)
outfile=open( "raw_captions2", "wb" )
dill.dump(d, outfile)
outfile.close()  
#url="http://www.newyorksocialdiary.com/party-pictures/2014/public-celebrations"
##a=caption_list(url)
#a

10
20
30
http://www.newyorksocialdiary.com/nysd/partypictures
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
http://www.newyorksocialdiary.com/party-pictures/2012/simple-pleasures
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190


C:\Users\Udi\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [6]:
 def caption_list2(url):
    x=0
    while x<10: # Trying 10 max for each URL before exiting
        response = requests.get(url)
        if response.status_code==200:
            break
        else:
            x+=1
    if x==10:
        return []
    
    soup = BeautifulSoup(response.text)
    captions=[]
    parent_div = soup.find('div',attrs={'class':'panel-pane pane-page-content'})
    if parent_div is None:
        return []
    
    for caption_div in parent_div.select('div.photocaption'):
        captions.append(caption_div.string)
    
    for caption_td in parent_div.select('td.photocaption'):
        captions.append(caption_td.string)
        
    for fontfrag in soup.select('font[size="1"]'):
            if fontfrag.string is None:
                continue
            captions.append(fontfrag.string.strip())
    if len(captions)>0:
        return captions
    else:
        print url
        return []
    return captions

In [658]:
url="http://www.newyorksocialdiary.com/party-pictures/2013/invitation-only-previews-0"

print url
response = requests.get(url)
soup=BeautifulSoup(response.text)
#captions = soup.findAll("td", {"class": "photocaption","scope":"row"})
#captions = soup.findAll("font", {"face": "Verdana, Arial, Helvetica, sans-serif"})
captions = soup.select('div[class=photocaption]')
captions = soup.select('td[class=photocaption]')
#captions=soup.findAll("div", {"class":["class1", "class2"]})
caption_list=[captions[i].text for i in range (len(captions))]
caption_list


http://www.newyorksocialdiary.com/party-pictures/2013/invitation-only-previews-0


[u'',
 u" Pamela Cohen, Nick Korniloff, Henri Barguirdjian, Steve Bernstein, Bob Chaloner, Mayor Mark Epley, Brenda Simons, and Richard Yasztremski at  last night's preview reception of Art Southampton.",
 u'',
 u'',
 u'Dietrich Klinge sculpture on the grounds of f Art Southampton',
 u'',
 u'',
 u'',
 u'Nick Korniloff and Pamela Cohen with the Tansey family',
 u'',
 u'Kevin Berlin and his aliens',
 u'',
 u'Models with Graff Jewelry and Perrier Jouet',
 u'',
 u'Ann Thomas',
 u'',
 u'Nancy Stone and Patty Kaus',
 u'',
 u'Katherine and Gary Andreassen',
 u'',
 u'Michel Witmer',
 u'',
 u'Henry Buhl, Pamela Cohen, and Nick Korniloff',
 u'',
 u'Guy and Dorothy Wiggins',
 u'',
 u'Jean Shafiroff and Ike Ude',
 u'',
 u'Joe Pintauro',
 u'',
 u'Amy and George Kane',
 u'',
 u'Nathan Joseph, Orlan, Rafael Cuir, and Stanley Cohen',
 u'',
 u'Brett McMicking, Ann Van Ness, and Michael Hall',
 u'',
 u'Cricket Burns',
 u'',
 u'Sue Madonia and Henry Koehler',
 u'',
 u'Michael Shaheen and Maryann Horwath'

In [5]:
outfile=open( "raw_captions", "r" )
d=pickle.load(outfile)
outfile.close()  


In [13]:
a=d[:]

In [14]:
#Removing empty and very long captions
a = filter(None, a)
a = [a[i] for i in range (len(a))  if len(a[i])<=250]
len(a)

74798

In [15]:
b=a[:]
#b

In [16]:
#Converting connection words to ",
for i, caption in enumerate(b):
    b[i]=caption.replace("and",",")
for i, caption in enumerate(b):
    b[i]=caption.replace(";",",")
for i, caption in enumerate(b):
    b[i]=caption.replace("&",",")
for i, caption in enumerate(b):
    b[i]=caption.replace("with",",")
for i, caption in enumerate(b):
    b[i]=caption.replace('"',",")
for i, caption in enumerate(b):
    b[i]=caption.replace('/',",")

In [17]:
#Replacing white space with ","
#for i, caption in enumerate(b):
#    b[i]=caption.replace(" ",",")
len(b)

74798

In [18]:
#Splitting into list of lists
for i, caption in enumerate(b):
    b[i]=caption.split(",")
#c=reduce(lambda x, y: x+y, b)
#c


In [19]:
e=b[:]


In [20]:
#removing private names only and some other one-word trash
for i, nest in enumerate(e):
    for j, person in enumerate(nest):
        #print person, len(person.split())
        if len(person.split())<=1:
            #print i,j
            e[i][j]="xxxxxx"


In [21]:
#removing white spaces on both sides
for i, nest in enumerate(e):
    for j, person in enumerate(nest):
        #print person, len(person.split())
        e[i][j]=person.strip()


In [22]:
#removing caption with only one person
#r=e[:]

for i, caption in enumerate(e):
    if len(caption)<=1:
        e[i]=['xxxxxx']
        #del(e[i])


In [23]:
#removing xxxxxxx externally
g=e[:]
while ['xxxxxx'] in g:
    g.remove(['xxxxxx'])


[[u'The scene at   IDEAL School', u'Academy\u2019s  10th\xa0Annual Gala.'],
 [u'Les Lieberman',
  u'Barri Lieberman',
  u'Isabel Kallman',
  u'Trish Iervolino',
  'xxxxxx',
  u'Ron Iervolino'],
 [u'Diana Rosario',
  u'Ali Sussman',
  u'Sarah Boll',
  u'Jen Zaleski',
  u'Alysse Brennan',
  'xxxxxx',
  u'Lindsay Macbeth'],
 ['xxxxxx', u'Tom Murro'],
 [u'Ron Iervolino',
  u'Trish Iervolino',
  u'Russ Middleton',
  'xxxxxx',
  u'Lisa Middleton'],
 [u'Barbara Loughlin', u'Dr. Gerald Loughlin', 'xxxxxx', u'Debbie Gelston'],
 [u'Heather Robinson',
  u'Kiwan Nichols',
  u'Jimmy Nichols',
  u'Melanie Carbone',
  'xxxxxx',
  u'Nancy Brown'],
 [u'Bill Mack', u'Les Lieberman'],
 [u'Dr. David Lyden', u'Patricia Sorenson'],
 [u'Jimmy Cayne', u'Vince Tese', 'xxxxxx', u'Pat Cayne'],
 [u'Stuart Oran', u'Les Lieberman', 'xxxxxx', u'Hilary Oran'],
 [u'Vince Tese', u'Chuck Grodin'],
 [u'Dwight Gooden', u'Les Lieberman'],
 [u'Dr. Amy Cunningham-Bussel', u'Ray Mirra', 'xxxxxx', u'Dr. Tyler Janovitz'],
 [u'D

In [24]:
#removing xxxxxx from within the lists
h=g[:]
for i, captions in enumerate(h):
    while 'xxxxxx' in captions:
        g[i].remove('xxxxxx')


In [25]:
unique_names=reduce(lambda x, y: x+y, h)
unique_names=list(set(unique_names))
len(unique_names)

76310

In [26]:
h

[[u'The scene at   IDEAL School', u'Academy\u2019s  10th\xa0Annual Gala.'],
 [u'Les Lieberman',
  u'Barri Lieberman',
  u'Isabel Kallman',
  u'Trish Iervolino',
  u'Ron Iervolino'],
 [u'Diana Rosario',
  u'Ali Sussman',
  u'Sarah Boll',
  u'Jen Zaleski',
  u'Alysse Brennan',
  u'Lindsay Macbeth'],
 [u'Tom Murro'],
 [u'Ron Iervolino', u'Trish Iervolino', u'Russ Middleton', u'Lisa Middleton'],
 [u'Barbara Loughlin', u'Dr. Gerald Loughlin', u'Debbie Gelston'],
 [u'Heather Robinson',
  u'Kiwan Nichols',
  u'Jimmy Nichols',
  u'Melanie Carbone',
  u'Nancy Brown'],
 [u'Bill Mack', u'Les Lieberman'],
 [u'Dr. David Lyden', u'Patricia Sorenson'],
 [u'Jimmy Cayne', u'Vince Tese', u'Pat Cayne'],
 [u'Stuart Oran', u'Les Lieberman', u'Hilary Oran'],
 [u'Vince Tese', u'Chuck Grodin'],
 [u'Dwight Gooden', u'Les Lieberman'],
 [u'Dr. Amy Cunningham-Bussel', u'Ray Mirra', u'Dr. Tyler Janovitz'],
 [u'Dan Shedrick', u'Samara Heafitz'],
 [u'Jason Adelman'],
 [u'Bart Scott', u'Mark Lapl'],
 [u'Mitch Rubin',

In [27]:
#creating caption dictionary
d1={}
for key,value in enumerate(h):
    d1[key]=value


In [28]:
d1

{0: [u'The scene at   IDEAL School', u'Academy\u2019s  10th\xa0Annual Gala.'],
 1: [u'Les Lieberman',
  u'Barri Lieberman',
  u'Isabel Kallman',
  u'Trish Iervolino',
  u'Ron Iervolino'],
 2: [u'Diana Rosario',
  u'Ali Sussman',
  u'Sarah Boll',
  u'Jen Zaleski',
  u'Alysse Brennan',
  u'Lindsay Macbeth'],
 3: [u'Tom Murro'],
 4: [u'Ron Iervolino',
  u'Trish Iervolino',
  u'Russ Middleton',
  u'Lisa Middleton'],
 5: [u'Barbara Loughlin', u'Dr. Gerald Loughlin', u'Debbie Gelston'],
 6: [u'Heather Robinson',
  u'Kiwan Nichols',
  u'Jimmy Nichols',
  u'Melanie Carbone',
  u'Nancy Brown'],
 7: [u'Bill Mack', u'Les Lieberman'],
 8: [u'Dr. David Lyden', u'Patricia Sorenson'],
 9: [u'Jimmy Cayne', u'Vince Tese', u'Pat Cayne'],
 10: [u'Stuart Oran', u'Les Lieberman', u'Hilary Oran'],
 11: [u'Vince Tese', u'Chuck Grodin'],
 12: [u'Dwight Gooden', u'Les Lieberman'],
 13: [u'Dr. Amy Cunningham-Bussel', u'Ray Mirra', u'Dr. Tyler Janovitz'],
 14: [u'Dan Shedrick', u'Samara Heafitz'],
 15: [u'Jason 

In [29]:
len(d1)

58737

In [48]:
pp={}
ppf={}
count=0
for p in range(len(d1)):
    name=unique_names[p]
    pp[name]=[]
    ppf[name]=[]
    #print name
    for i in range(len (d1)):
        if name in d1[i]:
            friends=d1[i]
            #print d[i]
            ppf[name].extend(friends)
    pp[name]=set(ppf[name])








In [49]:
#for i in len(pp):
pp1={}
for key in pp.keys():
    pp1[key]=len(pp[key])-1


In [112]:
from operator import itemgetter
pp2=sorted(pp1.iteritems(), key=itemgetter(1), reverse=True)
pp2

[(u'Jean Shafiroff', 292),
 (u'Mark Gilbertson', 271),
 (u'Gillian Miniter', 178),
 (u'Geoffrey Bradfield', 159),
 (u'Alina Cho', 150),
 (u'Somers Farkas', 142),
 (u'Kamie Lightburn', 140),
 (u'Andrew Saffir', 134),
 (u'Lucia Hwong Gordon', 132),
 (u'Yaz Hern', 131),
 (u'Debbie Bancroft', 129),
 (u'Sharon Bush', 128),
 (u'Lydia Fenet', 124),
 (u'Eleanora Kennedy', 122),
 (u'Bettina Zilkha', 112),
 (u'Kipton Cronkite', 112),
 (u'Jamee Gregory', 111),
 (u'Muffie Potter Aston', 109),
 (u'Karen LeFrak', 106),
 (u'Audrey Gruss', 106),
 (u'Deborah Norville', 104),
 (u'Leonard Lauder', 104),
 (u'Adelina Wong Ettelson', 98),
 (u'Liz Peek', 98),
 (u'Barbara Tober', 98),
 (u'Amy Fine Collins', 95),
 (u'Lisa Anastos', 95),
 (u'Grace Meigher', 91),
 (u'Nicole Miller', 91),
 (u'a friend', 91),
 (u'Fern Mallis', 90),
 (u'Martha Glass', 89),
 (u'Dayssi Olarte de Kanavos', 89),
 (u'Margaret Russell', 88),
 (u'Karen Klopp', 88),
 (u'Nathalie Kaplan', 87),
 (u'a Kellogg', 85),
 (u'Wendy Carduner', 85),


In [51]:
pd.Series([i[1] for i in pp2]).describe()*2

count    117474.000000
mean          6.968589
std          11.717051
min           0.000000
25%           2.000000
50%           4.000000
75%           8.000000
max         584.000000
dtype: float64

In [52]:
def bff(friends):
    d2={}
    unique_friends=list(set(friends))
    for name in unique_friends:
          d2[name]=friends.count(name)
    return d2


In [53]:
bff_dict={}
for name in ppf.keys():
    bff_dict[name]=bff(ppf[name])
bff_dict

{u'Tom Smarr': {u'Barbara Paul Robinson': 1,
  u'Claudia Bonn': 1,
  u'Louis Bauer': 1,
  u'Marechal Brown': 1,
  u'Mark Fisher': 1,
  u'Tom Smarr': 1},
 u'Christie Fraser': {u'Christie Fraser': 1, u'Liz Celeste': 1},
 u'Dennis Serras': {u'Dennis Serras': 1,
  u'Marlo Bakst': 1,
  u'Mikhail Baryshnikov': 1,
  u'Tara Nasta': 1},
 u'Honorees: Richard C. Hunt': {u'Andy Jagoda': 1,
  u'Honorees: Richard C. Hunt': 1,
  u'Kevin W. Chason': 1,
  u'Susan C. Waltman': 1},
 u'Samuel Barnett': {u'Jano Herbosch': 1,
  u'Joseph Timms': 1,
  u'Samuel Barnett': 1},
 u'Jennifer Lantzas': {u'Ashley Ludwig': 1,
  u'Jennifer Lantzas': 1,
  u'Jonathan Kuhn': 1,
  u'Will Ryman': 1},
 u'Pearson Chodak': {u'Jonathan Sheffer': 1, u'Pearson Chodak': 1},
 u'Sarah Julig': {u'Sarah Julig': 1, u'Tom Finkelpearl': 1},
 u'Jackie Terrebonne': {u'Jackie Terrebonne': 1,
  u'Jolene Travis': 1,
  u'Lee Cavanaugh': 1},
 u'Jaime E. Jim\xe9nez of Baccarat': {u'Chair Clare McKeon': 1,
  u'Chair Sloan Overstrom': 1,
  u'Jaime

In [54]:
ans3={}
for person in bff_dict.keys():
    for friend in bff_dict[person].keys():
            if person!=friend:
                ans3[person+':'+friend]=bff_dict[person][friend]
ans3

{u'Craig Phillip:Beat the Odds recipients  Moke Hein': 1,
 u'Caroline Williamson:Friederike Biggs': 2,
 u'Johannes Huebl:Greg Chan': 1,
 u'Livvi Luttrell:Helen Brock': 1,
 u'John Overbay:K. C. Anthony': 1,
 u'Dayssi Olarte de Kanavos:Amy Fine Collins': 1,
 u'Jenny  Toth:Courtney Hejl': 1,
 u'Marvin Davidson:Dudley Moore': 1,
 u'Jaclyn Scerbak:CJ de Nicola': 1,
 u'Shanette Barth Cohen:Press Officer of the Hampton Classic': 1,
 u"Lola Warbshey:Dr.  D'Angelo": 1,
 u'Yesim Philip:Susan Tabak': 1,
 u'Peter Lyden:a Anderson': 1,
 u'Richard Haray:Senior Vice President of Corporate Services': 1,
 u'Alex Langlois:Ian Mactaggart': 1,
 u'Mark Sanchez:Vice President of Special Events for Benefit Fund': 1,
 u'Colette Karnovsky:Nicholas Kalikow': 1,
 u'Melissa Lewis:Mildred Brinn': 1,
 u'Tommy Tickle Mouse:Leigh Brown': 1,
 u'Lorraine Schacht:Midge Laguardia': 1,
 u'Yumiko Honda:Robert Posner': 1,
 u'John Royall:Emily Graham': 1,
 u'Amara Toure:Shavonn Wheeler': 1,
 u'Rob Bristow:Alexis Halpern': 1,

In [69]:
from operator import itemgetter
ans4=sorted(ans3.iteritems(), key=itemgetter(1), reverse=True)
pd.Series([i[1] for i in ans4[:100]]).describe()

count    100.000000
mean      17.290000
std        9.641341
min       10.000000
25%       12.000000
50%       13.000000
75%       18.250000
max       54.000000
dtype: float64

In [118]:
import networkx as nx
G=nx.Graph()
for i in ans4:
    edges=i[0].split(':')
    G.add_edge(edges[0],edges[1])
len(G.edges())
#ans4[1][0].split(':')


125773

In [131]:
prank=nx.pagerank(G)

In [133]:
prank1=sorted(prank.iteritems(), key=itemgetter(1), reverse=True)
prank1
pd.Series([i[1] for i in prank1[:100]]).describe()

count    100.000000
mean       0.000198
std        0.000073
min        0.000135
25%        0.000150
50%        0.000180
75%        0.000215
max        0.000588
dtype: float64